<a href="https://colab.research.google.com/github/jfzhang726/finetune-reranker/blob/main/reranker_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BCE reranker finetuning


In [1]:
from google.colab import drive
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%bash
# git clone https://github.com/FlagOpen/FlagEmbedding.git
# cd FlagEmbedding
# pip install -e .

pip install -U FlagEmbedding

pip install LM_Cocktail

pip install accelerate

pip install transformers datasets

pip install faiss-gpu

# !pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
!pip install accelerate

In [ ]:
# how to use BCE reranker

# import torch
# from transformers import AutoModelForSequenceClassification, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
# model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
# model.eval()

# pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
# with torch.no_grad():
#     inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
#     scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
#     print(scores)

In [3]:
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import shutil
import os

## Finetuning

In [ ]:
# load dataset mteb/scidocs-reranking from huggingface
from datasets import load_dataset
dataset = load_dataset("mteb/scidocs-reranking")

# create a new dataset from the loaded dataset where the list of "positive" contains one positive sample randomly selected from the original "positive" list
import random
from datasets import DatasetDict, Dataset

def rename_keys(example):
  example['pos'] = example['positive']
  example.pop('positive')
  example['neg'] = example['negative']
  example.pop('negative')
  return example

def select_random_positive(example):
  example[f'pos'] = [random.choice(example['positive'])]
  example[f'neg'] = example.pop('negative')
  example.pop('positive')
  return example

def select_first_positive(example):
  example[f'pos'] = [example['positive'][0]]
  example[f'neg'] = example.pop('negative')
  example.pop('positive')
  return example



# new_dataset = DatasetDict({
#     split_name: dataset[split_name].map(select_first_positive, num_proc=4) for split_name in dataset})

new_dataset = DatasetDict({
    split_name: dataset[split_name].map(rename_keys, num_proc=4) for split_name in dataset})

new_dataset['validation'].to_json("scidocs/validation.jsonl", orient="records", lines=True)
new_dataset['test'].to_json("scidocs/test.jsonl", orient="records", lines=True)
small_dataset = new_dataset['validation'].select(range(256))
small_dataset.to_json("scidocs/small_validation.jsonl", orient="records", lines=True)

negative_numbers = [len(negative_list) for negative_list in new_dataset['validation']['neg']]
min_negative_number = min(negative_numbers)
max_negative_number = max(negative_numbers)
print(min_negative_number, max_negative_number)

train_group_size = min_negative_number + 1
print(train_group_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating validation split:   0%|          | 0/3981 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3978 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3981 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3978 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

22 49
23


''' bash
%%bash
torchrun --nproc_per_node 1 \
-m FlagEmbedding.reranker.run \
--output_dir models/finetune/ \
--model_name_or_path BAAI/bge-reranker-base \
--train_data ./scidocs/small_validation.jsonl \
--learning_rate 6e-5 \
--fp16 \
--num_train_epochs 5 \
--per_device_train_batch_size 16 \
--gradient_accumulation_steps 4 \
--dataloader_drop_last True \
--train_group_size 16 \
--max_len 512 \
--weight_decay 0.01 \
--logging_steps 10
'''

In [4]:



# Define the paths and other parameters

model_variation = "scidocs_3k_all_pos_20epoch"
train_data = "./scidocs/validation.jsonl"
output_dir = f"models/{model_variation}"

# Create a directory in Google Drive to save the model files
drive_output_dir = f"/content/drive/My Drive/Colab Notebooks/reranker/models/{model_variation}/"

output_dir_finetuned = os.path.join(output_dir, "finetuned")
output_dir_merged = os.path.join(output_dir, "merged")

drive_output_dir_finetuned = os.path.join(drive_output_dir, "finetuned")
drive_output_dir_merged = os.path.join(drive_output_dir, "merged")

model_name_or_path = "BAAI/bge-reranker-base"
learning_rate = 6e-5
num_train_epochs = 20
per_device_train_batch_size = 8 #16
gradient_accumulation_steps = 8 #4
max_len = 512
weight_decay = 0.01
logging_steps = 10

# Construct the command using string formatting
command = f"""
torchrun --nproc_per_node 1 \
-m FlagEmbedding.reranker.run \
--output_dir {output_dir_finetuned} \
--model_name_or_path {model_name_or_path} \
--train_data {train_data} \
--learning_rate {learning_rate} \
--fp16 \
--num_train_epochs {num_train_epochs} \
--per_device_train_batch_size {per_device_train_batch_size} \
--gradient_accumulation_steps {gradient_accumulation_steps} \
--dataloader_drop_last True \
--train_group_size 16 \
--max_len {max_len} \
--weight_decay {weight_decay} \
--logging_steps {logging_steps}
"""



In [ ]:
# Execute the command
!{command}

2024-06-12 11:16:26.141589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 11:16:26.141642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 11:16:26.143067: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-12 11:16:26.150191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 11:16:27.238124: W tensorflow/compiler/tf2

## Save model to google drive

In [ ]:

import shutil
import os





# Copy the entire output directory to Google Drive
shutil.copytree(output_dir_finetuned, drive_output_dir_finetuned)

print(f"Model files are saved to {drive_output_dir_finetuned}")

Model files are saved to /content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/finetuned


## Load model from google drive

In [ ]:

import torch

# Check if a GPU is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(drive_output_dir_finetuned)
model = AutoModelForSequenceClassification.from_pretrained(drive_output_dir_finetuned)
model.to(device)

# Use the model as needed
model.eval()

pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([-7.7553,  8.1673])


  Preparing metadata (setup.py) ... done
  Created wheel for LM_Cocktail: filename=LM_Cocktail-0.0.4-py3-none-any.whl size=9625 sha256=4ee94ac5ce93207d7fd3b48903ffcea206296424cee56203729bfe7fe88b61a6
  Stored in directory: /root/.cache/pip/wheels/5a/a3/04/d5d94c9b62efe10b261cf7946f9b1292ce709b7d2360363fca
Successfully built LM_Cocktail


## merged finetuned model with original model

In [ ]:
from LM_Cocktail import mix_models, mix_models_with_data

# Mix fine-tuned model and base model; then save it to output_path: ./mixed_model_1
model = mix_models(
    model_names_or_paths=["BAAI/bge-reranker-base", output_dir_finetuned],
    model_type='reranker',
    weights=[0.5, 0.5],  # you can change the weights to get a better trade-off.
    output_path=output_dir_merged)

loading BAAI/bge-reranker-base -----------------


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


loading models/scidocs_3k_all_pos_20epoch/finetuned -----------------
***weight for each model***: 
BAAI/bge-reranker-base 0.5
models/scidocs_3k_all_pos_20epoch/finetuned 0.5
Saving the new model to models/scidocs_3k_all_pos_20epoch/merged


In [ ]:


# Copy the entire output directory to Google Drive
shutil.copytree(output_dir_merged, drive_output_dir_merged)

print(f"Model files are saved to {drive_output_dir_merged}")

Model files are saved to /content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/merged


In [ ]:
import torch

# Check if a GPU is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(drive_output_dir_merged)
model = AutoModelForSequenceClassification.from_pretrained(drive_output_dir_merged)
model.to(device)

# Use the model as needed
model.eval()

pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([-7.9102,  7.2300])


## Evaluation

### Evaluate using FlagEmbedding script

https://github.com/FlagOpen/FlagEmbedding/tree/master/examples/finetune#6-evaluate-model

In [7]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00


### prepare dataset

https://github.com/FlagOpen/FlagEmbedding/tree/master/examples/finetune#62-your-dataset

In [ ]:
eval_corpus_file = "scidocs/eval_corpus.jsonl"
eval_queries_file = "scidocs/eval_queries.jsonl"


# load dataset mteb/scidocs-reranking from huggingface
from datasets import load_dataset, Dataset
import itertools
dataset = load_dataset("mteb/scidocs-reranking")
test_dataset = dataset['test']
eval_corpus = test_dataset['positive'] + test_dataset['negative']
eval_corpus = list(itertools.chain(*eval_corpus))
eval_corpus = [{"content": text} for text in eval_corpus]
eval_corpus = Dataset.from_list(eval_corpus)
# # test on small dataset
# eval_corpus = eval_corpus.select(range(1000))
eval_corpus.to_json(eval_corpus_file, orient="records", lines=True)

eval_queries = [{'query': example['query'], 'positive': example['positive']} for example in test_dataset]
eval_queries = Dataset.from_list(eval_queries)
# # test on small dataset
# eval_queries = eval_queries.select(range(32))
eval_queries.to_json(eval_queries_file, orient="records", lines=True)


Creating json from Arrow format:   0%|          | 0/119 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

1795563

In [ ]:
#### Retrieval, without reranking

In [ ]:
!python -m FlagEmbedding.baai_general_embedding.finetune.eval_msmarco \
--encoder BAAI/bge-base-en-v1.5 \
--fp16 \
--add_instruction \
--k 100 \
--corpus_data ./scidocs/eval_corpus.jsonl \
--query_data ./scidocs/eval_queries.jsonl


2024-06-10 11:54:49.396601: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 11:54:49.449902: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 11:54:49.449952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 11:54:49.451771: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-10 11:54:49.460191: I tensorflow/core/platform/cpu_feature_guar

#### retrieval and reranking

In [ ]:
# import FlagEmbedding
# from FlagEmbedding import FlagModel

In [8]:
import faiss
import torch
import logging
import datasets
import numpy as np
from tqdm import tqdm
from typing import Optional
from dataclasses import dataclass, field
from transformers import HfArgumentParser
from FlagEmbedding import FlagModel
from FlagEmbedding import FlagReranker
import json
import os
from collections import OrderedDict

logger = logging.getLogger(__name__)


@dataclass
class Args:
    encoder: str = field(
        default="BAAI/bge-base-en-v1.5",
        metadata={'help': 'The encoder name or path.'}
    )
    fp16: bool = field(
        default=False,
        metadata={'help': 'Use fp16 in inference?'}
    )
    add_instruction: bool = field(
        default=False,
        metadata={'help': 'Add query-side instruction?'}
    )

    corpus_data: str = field(
        default="namespace-Pt/msmarco",
        metadata={'help': 'candidate passages'}
    )
    query_data: str = field(
        default="namespace-Pt/msmarco-corpus",
        metadata={'help': 'queries and their positive passages for evaluation'}
    )

    max_query_length: int = field(
        default=32,
        metadata={'help': 'Max query length.'}
    )
    max_passage_length: int = field(
        default=128,
        metadata={'help': 'Max passage length.'}
    )
    batch_size: int = field(
        default=256,
        metadata={'help': 'Inference batch size.'}
    )
    index_factory: str = field(
        default="Flat",
        metadata={'help': 'Faiss index factory.'}
    )
    k: int = field(
        default=100,
        metadata={'help': 'How many neighbors to retrieve?'}
    )

    save_embedding: bool = field(
        default=False,
        metadata={'help': 'Save embeddings in memmap at save_dir?'}
    )
    load_embedding: bool = field(
        default=False,
        metadata={'help': 'Load embeddings from save_dir?'}
    )
    save_path: str = field(
        default="embeddings.memmap",
        metadata={'help': 'Path to save embeddings.'}
    )
    save_metrics: bool = field(
        default=False,
        metadata={'help': 'Save metrics?'}
    )
    save_metrics_path: str = field(
        default="./",
        metadata={'help': 'Path to save metrics.'}
    )
    save_retrieval_results: bool = field(
        default=False,
        metadata={'help': 'Save retrieval results?'}
    )
    save_retrieval_results_path: str = field(
        default="./",
        metadata={'help': 'Path to save retrieval results.'}
    )
    save_reranking_results: bool = field(
        default=False,
        metadata={'help': 'Save reranking results?'}
    )
    save_reranking_results_path: str = field(
        default="./",
        metadata={'help': 'Path to save reranking results.'}
    )



def index(model: FlagModel, corpus: datasets.Dataset, batch_size: int = 256, max_length: int=512, index_factory: str = "Flat", save_path: str = None, save_embedding: bool = False, load_embedding: bool = False):
    """
    1. Encode the entire corpus into dense embeddings;
    2. Create faiss index;
    3. Optionally save embeddings.
    """
    if load_embedding:
        test = model.encode("test")
        dtype = test.dtype
        dim = len(test)

        corpus_embeddings = np.memmap(
            save_path,
            mode="r",
            dtype=dtype
        ).reshape(-1, dim)

    else:
        corpus_embeddings = model.encode_corpus(corpus["content"], batch_size=batch_size, max_length=max_length)
        dim = corpus_embeddings.shape[-1]

        if save_embedding:
            logger.info(f"saving embeddings at {save_path}...")
            memmap = np.memmap(
                save_path,
                shape=corpus_embeddings.shape,
                mode="w+",
                dtype=corpus_embeddings.dtype
            )

            length = corpus_embeddings.shape[0]
            # add in batch
            save_batch_size = 10000
            if length > save_batch_size:
                for i in tqdm(range(0, length, save_batch_size), leave=False, desc="Saving Embeddings"):
                    j = min(i + save_batch_size, length)
                    memmap[i: j] = corpus_embeddings[i: j]
            else:
                memmap[:] = corpus_embeddings

    # create faiss index
    faiss_index = faiss.index_factory(dim, index_factory, faiss.METRIC_INNER_PRODUCT)

    if model.device == torch.device("cuda"):
        # co = faiss.GpuClonerOptions()
        co = faiss.GpuMultipleClonerOptions()
        co.useFloat16 = True
        # faiss_index = faiss.index_cpu_to_gpu(faiss.StandardGpuResources(), 0, faiss_index, co)
        faiss_index = faiss.index_cpu_to_all_gpus(faiss_index, co)

    # NOTE: faiss only accepts float32
    logger.info("Adding embeddings...")
    corpus_embeddings = corpus_embeddings.astype(np.float32)
    faiss_index.train(corpus_embeddings)
    faiss_index.add(corpus_embeddings)
    return faiss_index


def search(model: FlagModel, queries: datasets, faiss_index: faiss.Index, k:int = 100, batch_size: int = 256, max_length: int=512):
    """
    1. Encode queries into dense embeddings;
    2. Search through faiss index
    """
    query_embeddings = model.encode_queries(queries["query"], batch_size=batch_size, max_length=max_length)
    query_size = len(query_embeddings)

    all_scores = []
    all_indices = []

    for i in tqdm(range(0, query_size, batch_size), desc="Searching"):
        j = min(i + batch_size, query_size)
        query_embedding = query_embeddings[i: j]
        score, indice = faiss_index.search(query_embedding.astype(np.float32), k=k)
        all_scores.append(score)
        all_indices.append(indice)

    all_scores = np.concatenate(all_scores, axis=0)
    all_indices = np.concatenate(all_indices, axis=0)
    return all_scores, all_indices


def rerank(model_name_or_path, queries, retrieval_results):
    reranking_pairs = []
    for i in range(len(queries)):
      pairs_for_query = []
      for j in range(len(retrieval_results[i])):
        pairs_for_query.append((queries[i]['query'], retrieval_results[i][j]))
      reranking_pairs.append(pairs_for_query)

    reranking_scores = []



    # Check if a GPU is available and move the model to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path)
    model.to(device)

    # Use the model as needed
    model.eval()
    with torch.no_grad():
      for i in tqdm(range(0, len(reranking_pairs)), desc="reranking"):
        pairs = reranking_pairs[i]
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
        # print(scores)
        reranking_scores.append(scores.tolist())

    retrieval_results_sorted = []
    for i in range(len(reranking_pairs)):
      combined = list(zip(reranking_pairs[i], reranking_scores[i]))
      sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)
      retrieval_results_sorted.append(sorted_combined)

    reranking_results = []
    for i in range(len(retrieval_results_sorted)):
      reranking_results.append([x[0][1] for x in retrieval_results_sorted[i]])
    return reranking_results, retrieval_results_sorted


def rerank_flag_model(model_name_or_path, queries, retrieval_results):
    reranking_pairs = []
    for i in range(len(queries)):
      pairs_for_query = []
      for j in range(len(retrieval_results[i])):
        pairs_for_query.append((queries[i]['query'], retrieval_results[i][j]))
      reranking_pairs.append(pairs_for_query)

    reranking_scores = []

    reranker = FlagReranker(model_name_or_path, use_fp16=True) #use fp16 can speed up computing
    for i in tqdm(range(0, len(reranking_pairs)), desc="reranking"):
        pairs = reranking_pairs[i]
        scores = reranker.compute_score(pairs)
        reranking_scores.append(scores)
        retrieval_results_sorted = []


    for i in range(len(reranking_pairs)):
      combined = list(zip(reranking_pairs[i], reranking_scores[i]))
      sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)
      retrieval_results_sorted.append(sorted_combined)

    reranking_results = []
    for i in range(len(retrieval_results_sorted)):
      reranking_results.append([x[0][1] for x in retrieval_results_sorted[i]])
    return reranking_results, retrieval_results_sorted


def evaluate(preds, labels, cutoffs=[1,10,100]):
    """
    Evaluate MRR and Recall at cutoffs.
    """
    metrics = {}

    # MRR
    mrrs = np.zeros(len(cutoffs))
    for pred, label in zip(preds, labels):
        jump = False
        for i, x in enumerate(pred, 1):
            if x in label:
                for k, cutoff in enumerate(cutoffs):
                    if i <= cutoff:
                        mrrs[k] += 1 / i
                jump = True
            if jump:
                break
    mrrs /= len(preds)
    for i, cutoff in enumerate(cutoffs):
        mrr = mrrs[i]
        metrics[f"MRR@{cutoff}"] = mrr

    # Recall
    recalls = np.zeros(len(cutoffs))
    for pred, label in zip(preds, labels):
        for k, cutoff in enumerate(cutoffs):
            recall = np.intersect1d(label, pred[:cutoff])
            recalls[k] += len(recall) / len(label)
    recalls /= len(preds)
    for i, cutoff in enumerate(cutoffs):
        recall = recalls[i]
        metrics[f"Recall@{cutoff}"] = recall

    return metrics


def main(args):
    # parser = HfArgumentParser([Args])
    # args: Args = parser.parse_args_into_dataclasses()[0]

    if args.query_data == 'namespace-Pt/msmarco-corpus':
        assert args.corpus_data == 'namespace-Pt/msmarco'
        eval_data = datasets.load_dataset("namespace-Pt/msmarco", split="dev")
        corpus = datasets.load_dataset("namespace-Pt/msmarco-corpus", split="train")
    else:
        eval_data = datasets.load_dataset('json', data_files=args.query_data, split='train')
        corpus = datasets.load_dataset('json', data_files=args.corpus_data, split='train')

    all_metrics = OrderedDict()
    print("loading embedding model")
    model = FlagModel(
        args.encoder,
        query_instruction_for_retrieval="Represent this sentence for searching relevant passages: " if args.add_instruction else None,
        use_fp16=args.fp16
    )
    print("building index")
    faiss_index = index(
        model=model,
        corpus=corpus,
        batch_size=args.batch_size,
        max_length=args.max_passage_length,
        index_factory=args.index_factory,
        save_path=args.save_path,
        save_embedding=args.save_embedding,
        load_embedding=args.load_embedding
    )
    print("retrieval")
    scores, indices = search(
        model=model,
        queries=eval_data,
        faiss_index=faiss_index,
        k=args.k,
        batch_size=args.batch_size,
        max_length=args.max_query_length
    )

    retrieval_results = []
    for indice in indices:
        # filter invalid indices
        indice = indice[indice != -1].tolist()
        retrieval_results.append(corpus[indice]["content"])

    ground_truths = []
    for sample in eval_data:
        ground_truths.append(sample["positive"])


    metrics = evaluate(retrieval_results, ground_truths)
    all_metrics["retrieval"] = metrics
    print(metrics)
    if args.save_retrieval_results:
      with open(os.path.join(args.save_retrieval_results_path, "retrieval_results.json"), "w") as f:
        json.dump(retrieval_results, f, indent=4)


    # Load reranking tokenizer and model
    print("BGE reranking")
    # tokenizer_reranking = AutoTokenizer.from_pretrained('BAAI/bge-reranker-base')
    # model_reranking = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-base')
    reranking_results, retrieval_results_sorted = rerank_flag_model(model_name_or_path='BAAI/bge-reranker-base',
                               queries=eval_data,
                               retrieval_results=retrieval_results)

    metrics = evaluate(reranking_results, ground_truths)
    all_metrics["bge_reranking"] = metrics

    print(metrics)
    if args.save_reranking_results:
      with open(os.path.join(args.save_reranking_results_path, "bge_reranking_results.json"), "w") as f:
        json.dump(reranking_results, f, indent=4)
      with open(os.path.join(args.save_reranking_results_path, "bge_reranking_results_sorted.json"), "w") as f:
        json.dump(retrieval_results_sorted, f, indent=4)


    print("finetuned reranking")
    # tokenizer_reranking = AutoTokenizer.from_pretrained(drive_output_dir_finetuned)
    # model_reranking = AutoModelForSequenceClassification.from_pretrained(drive_output_dir_finetuned)
    reranking_results, retrieval_results_sorted = rerank_flag_model(model_name_or_path=drive_output_dir_finetuned,
                               queries=eval_data,
                               retrieval_results=retrieval_results)

    metrics = evaluate(reranking_results, ground_truths)
    all_metrics["finetuned_reranking"] = metrics
    print(metrics)
    if args.save_reranking_results:
      with open(os.path.join(args.save_reranking_results_path, "finetuned_reranking_results.json"), "w") as f:
        json.dump(reranking_results, f, indent=4)
      with open(os.path.join(args.save_reranking_results_path, "finetuned_reranking_results_sorted.json"), "w") as f:
        json.dump(retrieval_results_sorted, f, indent=4)


    print("merged reranking")
    # Load reranking tokenizer and model
    # tokenizer_reranking = AutoTokenizer.from_pretrained(drive_output_dir_merged)
    # model_reranking = AutoModelForSequenceClassification.from_pretrained(drive_output_dir_merged)
    reranking_results, retrieval_results_sorted = rerank_flag_model(model_name_or_path=drive_output_dir_merged,
                               queries=eval_data,
                               retrieval_results=retrieval_results)

    metrics = evaluate(reranking_results, ground_truths)
    all_metrics["merged_reranking"] = metrics
    print(metrics)
    if args.save_reranking_results:
      with open(os.path.join(args.save_reranking_results_path, "merged_reranking_results.json"), "w") as f:
        json.dump(reranking_results, f, indent=4)
      with open(os.path.join(args.save_reranking_results_path, "merged_reranking_results_sorted.json"), "w") as f:
        json.dump(retrieval_results_sorted, f, indent=4)

    print(all_metrics)
    if args.save_metrics:
      with open(os.path.join(args.save_metrics_path, "metrics.json"), "w") as f:
        json.dump(all_metrics, f, indent=4)


# if __name__ == "__main__":
#     main()



In [ ]:
os.makedirs("./scidocs/eval", exist_ok=True)

args = Args(
    encoder='BAAI/bge-base-en-v1.5',
    fp16=True,
    add_instruction=False,
    k=100,
    corpus_data="./scidocs/eval_corpus.jsonl",
    query_data="./scidocs/eval_queries.jsonl",
    save_metrics=True,
    save_metrics_path="./scidocs/eval/",
    save_retrieval_results=True,
    save_retrieval_results_path="./scidocs/eval/",
    save_reranking_results=True,
    save_reranking_results_path="./scidocs/eval/",
)

main(args)

In [9]:
args = Args(
    encoder='BAAI/bge-base-en-v1.5',
    fp16=True,
    add_instruction=False,
    k=100,
    save_metrics=False,
    save_retrieval_results=False,
    save_reranking_results=False,
)

main(args)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating dev split:   0%|          | 0/6980 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/8841823 [00:00<?, ? examples/s]

loading embedding model


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

building index


Inference Embeddings:   0%|          | 171/34539 [01:06<3:43:34,  2.56it/s]


KeyboardInterrupt: 

In [ ]:
# import shutil
# import os





# # Copy the entire output directory to Google Drive
# drive_output_dir_eval = os.path.join(drive_output_dir, 'eval')
# shutil.copytree("./scidocs", drive_output_dir_eval)

# print(f"Evaluation files are saved to {drive_output_dir_eval}")

FileExistsError: [Errno 17] File exists: '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k/eval'

In [ ]:
from distutils.dir_util import copy_tree

drive_output_dir_eval = os.path.join(drive_output_dir, 'eval')

source_dir = "./scidocs"
dest_dir = drive_output_dir_eval

# Copy source directory to destination directory
copy_tree(source_dir, dest_dir)

['/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval_corpus.jsonl',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/finetuned_reranking_results.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/retrieval_results.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/bge_reranking_results_sorted.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/bge_reranking_results_metrics.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/bge_reranking_results.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/retrieval_results_metrics.json',
 '/content/drive/My Drive/Colab Notebooks/reranker/models/scidocs_3k_all_pos_20epoch/eval/eval/merged_reranking_results.json'

In [ ]:
eval_data = datasets.load_dataset('json', data_files=args.query_data, split='train')
corpus = datasets.load_dataset('json', data_files=args.corpus_data, split='train')

model = FlagModel(
    args.encoder,
    query_instruction_for_retrieval="Represent this sentence for searching relevant passages: " if args.add_instruction else None,
    use_fp16=args.fp16
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

{'content': 'A modified particle swarm optimizer'}

In [ ]:
faiss_index = index(
      model=model,
      corpus=corpus,
      batch_size=args.batch_size,
      max_length=args.max_passage_length,
      index_factory=args.index_factory,
      save_path=args.save_path,
      save_embedding=args.save_embedding,
      load_embedding=args.load_embedding
  )

Inference Embeddings: 100%|██████████| 4/4 [00:07<00:00,  1.89s/it]


In [ ]:
scores, indices = search(
      model=model,
      queries=eval_data,
      faiss_index=faiss_index,
      k=args.k,
      batch_size=args.batch_size,
      max_length=args.max_query_length
  )

Searching: 100%|██████████| 1/1 [00:00<00:00, 271.55it/s]


In [ ]:
scores.shape, indices.shape

((32, 100), (32, 100))

In [ ]:
scores[0], indices[0]

(array([0.8388779 , 0.6963649 , 0.6940613 , 0.6855496 , 0.6843265 ,
        0.67818224, 0.67231023, 0.6699437 , 0.6597383 , 0.6531971 ,
        0.65202266, 0.64886904, 0.64692426, 0.64554596, 0.64155966,
        0.640073  , 0.640073  , 0.63916093, 0.637592  , 0.63732564,
        0.6370159 , 0.634825  , 0.63283134, 0.63168764, 0.63168764,
        0.6234373 , 0.6234373 , 0.6202165 , 0.61900693, 0.617807  ,
        0.61683464, 0.6159196 , 0.6157845 , 0.6142101 , 0.6137464 ,
        0.610267  , 0.6093395 , 0.60836834, 0.6074803 , 0.6048096 ,
        0.60406446, 0.60305184, 0.5996891 , 0.5985911 , 0.5973567 ,
        0.5960336 , 0.59502745, 0.59385663, 0.5929638 , 0.59160113,
        0.591112  , 0.58992076, 0.58958524, 0.5876996 , 0.5873552 ,
        0.5862504 , 0.5849861 , 0.579559  , 0.5784404 , 0.578367  ,
        0.57795954, 0.57688445, 0.5755788 , 0.5732786 , 0.5709024 ,
        0.57055247, 0.56969273, 0.5696851 , 0.569343  , 0.56929445,
        0.56927043, 0.5691583 , 0.56844926, 0.56

In [ ]:
retrieval_results = []
for indice in indices:
    # filter invalid indices
    indice = indice[indice != -1].tolist()
    retrieval_results.append(corpus[indice]["content"])

ground_truths = []
for sample in eval_data:
    ground_truths.append(sample["positive"])

In [ ]:
len(retrieval_results), len(retrieval_results[0]), len(ground_truths), [len(t) for t in ground_truths[:5]]

(32, 100, 32, [5, 5, 4, 5, 4])

In [ ]:
retrieval_results[0], ground_truths[0], eval_queries[0]

(['A Hybrid EP and SQP for Dynamic Economic Dispatch with Nonsmooth Fuel Cost Function',
  'Dynamic Programming and Optimal Control',
  'Differential Evolution-A simple and efficient adaptive scheme for global optimization over continuous spaces by',
  'Genetic Algorithms in Search Optimization and Machine Learning',
  'On the limited memory BFGS method for large scale optimization',
  'A modified particle swarm optimizer',
  'The particle swarm - explosion, stability, and convergence in a multidimensional complex space',
  'Hybrid Particle Swarm Optimiser with breeding and subpopulations',
  'Simulated annealing: Practice versus theory',
  'Online Convex Programming and Generalized Infinitesimal Gradient Ascent',
  'Introduction to Algorithms',
  'Logarithmic regret algorithms for online convex optimization',
  'A polynomial-time approximation scheme for maximizing the minimum machine completion time',
  'An overview of gradient descent optimization algorithms',
  'Maximum margin plan

In [ ]:
reranking_pairs = []
for i in range(len(eval_data)):
  pairs_for_query = []
  for j in range(len(retrieval_results[i])):
    pairs_for_query.append((eval_data[i]['query'], retrieval_results[i][j]))
  reranking_pairs.append(pairs_for_query)

reranking_scores = []


In [ ]:
len(reranking_pairs[0]), reranking_pairs[0]

(100,
 [('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'A Hybrid EP and SQP for Dynamic Economic Dispatch with Nonsmooth Fuel Cost Function'),
  ('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'Dynamic Programming and Optimal Control'),
  ('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'Differential Evolution-A simple and efficient adaptive scheme for global optimization over continuous spaces by'),
  ('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'Genetic Algorithms in Search Optimization and Machine Learning'),
  ('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'On the limited memory BFGS method for large scale optimization'),
  ('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'A modified particle swarm optimizer'),
  ('A Direct Search Method

In [ ]:
tokenizer_reranking = AutoTokenizer.from_pretrained('BAAI/bge-reranker-base')
model_reranking = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-base')



tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
reranking_scores = []

model = model_reranking
tokenizer = tokenizer_reranking
# Use the model as needed
model.eval()


with torch.no_grad():
  for i in tqdm(range(0, len(reranking_pairs)), desc="reranking"):
    pairs = reranking_pairs[i]
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    # print(scores)
    reranking_scores.append(scores.tolist())

reranking: 100%|██████████| 32/32 [05:19<00:00, 10.00s/it]


In [ ]:
reranking_scores[0]

[-4.256772041320801,
 -8.052172660827637,
 -10.188249588012695,
 -8.917725563049316,
 -7.397087574005127,
 -10.196633338928223,
 -10.195597648620605,
 -10.195937156677246,
 -10.024155616760254,
 -10.196907043457031,
 -7.629403114318848,
 -6.910417556762695,
 -10.143218040466309,
 -8.030284881591797,
 -10.19527530670166,
 -8.794960021972656,
 -8.794960021972656,
 -9.695181846618652,
 -9.764251708984375,
 -9.919110298156738,
 -9.918163299560547,
 -10.111714363098145,
 -10.196910858154297,
 -10.195643424987793,
 -10.195643424987793,
 -8.522886276245117,
 -8.522886276245117,
 -10.074542045593262,
 -8.070296287536621,
 -7.007521629333496,
 -10.195812225341797,
 -10.188712120056152,
 -9.771354675292969,
 -8.109670639038086,
 -10.19676399230957,
 -10.174168586730957,
 -7.827511310577393,
 -10.180580139160156,
 -10.1968994140625,
 -10.196664810180664,
 -2.2481935024261475,
 -9.508407592773438,
 -10.19474983215332,
 -10.196873664855957,
 -9.360511779785156,
 -6.528110027313232,
 -10.02785587310

In [ ]:
reranking_scores = []

from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-base', use_fp16=True) #use fp16 can speed up computing
for i in tqdm(range(0, len(reranking_pairs)), desc="reranking"):
    pairs = reranking_pairs[i]
    scores = reranker.compute_score(pairs)
    reranking_scores.append(scores)


reranking: 100%|██████████| 32/32 [00:02<00:00, 12.20it/s]


In [ ]:
len(reranking_scores[0]), reranking_scores[0]

In [ ]:
reranking_scores = []

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(drive_output_dir_finetuned)
model = AutoModelForSequenceClassification.from_pretrained(drive_output_dir_finetuned)

# Use the model as needed
model.eval()


with torch.no_grad():
  for i in tqdm(range(0, len(reranking_pairs)), desc="reranking"):
    pairs = reranking_pairs[i]
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    # print(scores)
    reranking_scores.append(scores.tolist())

reranking:  12%|█▎        | 4/32 [01:01<07:12, 15.45s/it]


KeyboardInterrupt: 

In [ ]:
reranking_scores[0]

[3.245600938796997,
 -9.324012756347656,
 -7.368051528930664,
 -2.8568203449249268,
 -8.228888511657715,
 -9.369584083557129,
 -9.599268913269043,
 -9.87158489227295,
 -9.096476554870605,
 -9.243561744689941,
 -6.345494270324707,
 -5.311014175415039,
 -7.993556022644043,
 -8.048674583435059,
 -8.760575294494629,
 -0.9795569181442261,
 -0.9795569181442261,
 -5.379862308502197,
 -7.545672416687012,
 -8.274022102355957,
 -9.075865745544434,
 -8.797242164611816,
 -9.117416381835938,
 -4.958551406860352,
 -4.958551406860352,
 -6.58682918548584,
 -6.58682918548584,
 -9.075919151306152,
 -7.4549102783203125,
 -7.49788761138916,
 -9.544417381286621,
 -8.566130638122559,
 -7.621795654296875,
 -8.289202690124512,
 -7.9222002029418945,
 -9.724967002868652,
 -8.386632919311523,
 -9.720623970031738,
 -8.685096740722656,
 -8.696930885314941,
 -4.890108585357666,
 -9.89018726348877,
 -7.538196563720703,
 -10.107608795166016,
 -7.06441593170166,
 -7.589984893798828,
 -8.701125144958496,
 -9.0518188476

In [ ]:
reranking_scores = []

from FlagEmbedding import FlagReranker
reranker = FlagReranker(drive_output_dir_finetuned, use_fp16=True) #use fp16 can speed up computing
for i in tqdm(range(0, len(reranking_pairs)), desc="reranking"):
    pairs = reranking_pairs[i]
    scores = reranker.compute_score(pairs)
    reranking_scores.append(scores)

reranking: 100%|██████████| 32/32 [00:02<00:00, 12.10it/s]


In [ ]:
retrieval_results_sorted = []
for i in range(len(reranking_pairs)):
  combined = list(zip(reranking_pairs[i], reranking_scores[i]))
  sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)
  retrieval_results_sorted.append(sorted_combined)

In [ ]:
retrieval_results_sorted[0]

[(('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'A Hybrid EP and SQP for Dynamic Economic Dispatch with Nonsmooth Fuel Cost Function'),
  3.245600938796997),
 (('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'Nonlinear Component Analysis as a Kernel Eigenvalue Problem'),
  -3.3776421546936035),
 (('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'Thumbs Up or Thumbs Down? Semantic Orientation Applied to Unsupervised Classification of Reviews'),
  -5.153554916381836),
 (('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'Stochastic Neighbor Embedding'),
  -5.564761161804199),
 (('A Direct Search Method to solve Economic Dispatch Problem with Valve-Point Effect',
   'Text mining of news-headlines for FOREX market prediction: A Multi-layer Dimension Reduction Algorithm with semantics and sentiment'),
  -6.089075565338135),
 (('A D

In [ ]:
reranking_results = []
for i in range(len(retrieval_results_sorted)):
  reranking_results.append([x[0][1] for x in retrieval_results_sorted[i]])

In [ ]:
metrics = evaluate(retrieval_results, ground_truths)

print(metrics)

{'MRR@1': 0.21875, 'MRR@10': 0.21875, 'MRR@100': 0.21875, 'Recall@1': 0.046875, 'Recall@10': 0.20625, 'Recall@100': 0.2125}


In [ ]:
metrics = evaluate(reranking_results, ground_truths)

print(metrics)

{'MRR@1': 0.21875, 'MRR@10': 0.21875, 'MRR@100': 0.21875, 'Recall@1': 0.046875, 'Recall@10': 0.16093749999999998, 'Recall@100': 0.2125}


In [ ]:
retrieval_results[0], reranking_results[0]

(['A Hybrid EP and SQP for Dynamic Economic Dispatch with Nonsmooth Fuel Cost Function',
  'A modified particle swarm optimizer',
  'A hybrid of genetic algorithm and particle swarm optimization for recurrent network design',
  'Identification and control of dynamic systems using recurrent fuzzy neural networks',
  'Concept Based Query Expansion',
  'Term-Weighting Approaches in Automatic Text Retrieval',
  'Challenges for efficient communication in underwater acoustic sensor networks',
  'Genetic Fuzzy Systems - Evolutionary Tuning and Learning of Fuzzy Knowledge Bases',
  'Adaptive Manifold Learning',
  'Text mining of news-headlines for FOREX market prediction: A Multi-layer Dimension Reduction Algorithm with semantics and sentiment',
  'Nonlinear Component Analysis as a Kernel Eigenvalue Problem',
  'Query by Committee',
  'The challenges of building mobile underwater wireless networks for aquatic applications',
  'Colorful Image Colorization',
  'Manifold-Learning-Based Feature Ex

### evaluate using ragas

In [ ]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
import tqdm
persist_directory = "./vectorstore/"
embedding = HuggingFaceEmbeddings(model_name="")